In [1]:
import pandas as pd
import numpy as np
from storeEOI import IscEoiStore

In [2]:
from ksql import KSQLAPI
import socket
from confluent_kafka import Consumer, KafkaError, KafkaException

In [3]:
from ksql_data_helpers import return_det_counts_data, return_recent_data_ts,ts_str_to_unix, ts_unix_to_ts
from helpers import return_det_waveform

In [4]:
import logging
logging.basicConfig(
            format='%(asctime)s %(levelname)-8s %(message)s',
            level=logging.ERROR,
            datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [5]:
client = KSQLAPI('http://localhost:8088')

In [6]:
isc_eoi = IscEoiStore()

In [7]:
# 1673227575000 

In [8]:
# 1673227160000

In [9]:
# 1673461500000  1673463000000

In [23]:

all_rows = isc_eoi.start_processing()
# print(f"all_isc_store")     
# print(isc_eoi.all_isc_store)

Empty DataFrame
Columns: [SIGNALID, WINDOWSTART, WINDOWEND, SUMFLAG, processed]
Index: []


In [24]:
all_rows

[]

In [22]:
insert_to_ksql_stream()

In [13]:
def insert_to_ksql_stream():
    all_res = []
    for each_signal, all_dets in  isc_eoi.all_isc_store.items():
        for each_det, interruptions in all_dets.items():
            if(interruptions):
                prev_timestamp = None
                base_vol = 0
                curr_vol = 0
                curr_ma = 0
                reduction = 0
                count =0
                for each_interruption in interruptions[::-1]:


                    if((prev_timestamp is None) ):

                        base_vol += each_interruption['baseline']
                        curr_vol += each_interruption['curr_volume']
                        reduction += each_interruption['reduction']
                        prev_timestamp = each_interruption['end_time']
                        count +=1
                    else:
                        time_diff = (pd.to_datetime(prev_timestamp) - pd.to_datetime(each_interruption['end_time'])).total_seconds()
                        print(f"det id {each_det} time diff {time_diff}")
                        if(time_diff == 300):
                            base_vol += each_interruption['baseline']
                            curr_vol += each_interruption['curr_volume']
                            reduction += each_interruption['reduction']
                            prev_timestamp = each_interruption['end_time']
                            count +=1

                res_dict ={}
                res_dict['SIGNALID'] = each_signal
                res_dict['REDUCTION'] = (reduction/count)*100
                res_dict['DURATION'] =300*count

                res_dict['STARTVOLUME'] = float(interruptions[0]['curr_volume'])
                res_dict['STARTBASELINE'] = interruptions[0]['baseline']
                res_dict['STARTMA'] = interruptions[0]['curr_ma']

                res_dict['TOTALVOLUME'] = float(curr_vol)
                res_dict['TOTALBASELINE'] = base_vol

                res_dict['TIMEEND'] = interruptions[0]['end_time']

                res_dict['DETECTORID'] = str(each_det)



                all_res.append(res_dict)
                client.inserts_stream('interruptionsdetector', [res_dict])
            else:
                res_dict = {}
                res_dict['DETECTORID'] = str(each_det)
                client.inserts_stream('interruptionsdetector', [res_dict])
    #             no interruptions for this detector. insert tombstone record


2023-01-10 21:21:33 INFO     Received unhandled event <RemoteSettingsChanged changed_settings:{ChangedSetting(setting=SettingCodes.MAX_CONCURRENT_STREAMS, original_value=None, new_value=100), ChangedSetting(setting=SettingCodes._max_header_list_size, original_value=None, new_value=8192)}>
2023-01-10 21:21:33 INFO     Received unhandled event <SettingsAcknowledged changed_settings:{ChangedSetting(setting=SettingCodes.ENABLE_PUSH, original_value=1, new_value=0)}>
2023-01-10 21:21:33 INFO     Received unhandled event <SettingsAcknowledged changed_settings:{}>
2023-01-10 21:21:33 INFO     Received unhandled event <PriorityUpdated stream_id:1, weight:16, depends_on:0, exclusive:False>
2023-01-10 21:21:33 INFO     Received unhandled event <RemoteSettingsChanged changed_settings:{ChangedSetting(setting=SettingCodes.MAX_CONCURRENT_STREAMS, original_value=None, new_value=100), ChangedSetting(setting=SettingCodes._max_header_list_size, original_value=None, new_value=8192)}>
2023-01-10 21:21:33 I

In [18]:

# tombstone test
client.inserts_stream('interruptionsdetector', [{'DETECTORID': '28702', 'valu"e':None}])

2023-01-10 21:19:43 INFO     Received unhandled event <RemoteSettingsChanged changed_settings:{ChangedSetting(setting=SettingCodes.MAX_CONCURRENT_STREAMS, original_value=None, new_value=100), ChangedSetting(setting=SettingCodes._max_header_list_size, original_value=None, new_value=8192)}>
2023-01-10 21:19:43 INFO     Received unhandled event <SettingsAcknowledged changed_settings:{ChangedSetting(setting=SettingCodes.ENABLE_PUSH, original_value=1, new_value=0)}>
2023-01-10 21:19:43 INFO     Received unhandled event <SettingsAcknowledged changed_settings:{}>
2023-01-10 21:19:43 INFO     Received unhandled event <PriorityUpdated stream_id:1, weight:16, depends_on:0, exclusive:False>


[{'seq': 0, 'status': 'ok'}]

In [19]:
client.inserts_stream('interruptionsdetector', all_res)

2023-01-10 21:19:49 INFO     Received unhandled event <RemoteSettingsChanged changed_settings:{ChangedSetting(setting=SettingCodes.MAX_CONCURRENT_STREAMS, original_value=None, new_value=100), ChangedSetting(setting=SettingCodes._max_header_list_size, original_value=None, new_value=8192)}>
2023-01-10 21:19:49 INFO     Received unhandled event <SettingsAcknowledged changed_settings:{ChangedSetting(setting=SettingCodes.ENABLE_PUSH, original_value=1, new_value=0)}>
2023-01-10 21:19:49 INFO     Received unhandled event <SettingsAcknowledged changed_settings:{}>
2023-01-10 21:19:49 INFO     Received unhandled event <PriorityUpdated stream_id:1, weight:16, depends_on:0, exclusive:False>


[{'seq': 0, 'status': 'ok'},
 {'seq': 1, 'status': 'ok'},
 {'seq': 2, 'status': 'ok'},
 {'seq': 3, 'status': 'ok'},
 {'seq': 4, 'status': 'ok'},
 {'seq': 5, 'status': 'ok'},
 {'seq': 6, 'status': 'ok'},
 {'seq': 7, 'status': 'ok'},
 {'seq': 8, 'status': 'ok'},
 {'seq': 9, 'status': 'ok'},
 {'seq': 10, 'status': 'ok'},
 {'seq': 11, 'status': 'ok'},
 {'seq': 12, 'status': 'ok'},
 {'seq': 13, 'status': 'ok'}]

In [133]:
res_dict

{'SIGNALID': '2870',
 'REDUCTION': 21.49532710280373,
 'DURATION': 300,
 'STARTVOLUME': 7.0,
 'STARTBASELINE': 8.916666666666666,
 'STARTMA': 7.75,
 'TOTALVOLUME': 7.0,
 'TOTALBASELINE': 8.916666666666666,
 'TIMEEND': '2023-01-10T19:54:59.999000',
 'DETECTORID': 28702}

In [110]:
res_dict

{'SIGNALID': '1010',
 'REDUCTION': 55.55555555555556,
 'DURATION': 300,
 'STARTVOLUME': 3,
 'STARTBASELINE': 6.75,
 'STARTMA': 6.75,
 'TOTALVOLUME': 3,
 'TOTALBASELINE': 6.75,
 'TIMEEND': '2023-01-10T19:59:59.999000',
 'DETECTORID': 10103}

In [106]:
pd.DataFrame(all_res)

,SIGNALID,REDUCTION,DURATION,STARTVOLUME,STARTBASELINE,STARTMA,TOTALVOLUME,TOTALBASELINE,TIMEEND,DETECTORID
0,1010,55.555556,300,3,6.750000,6.75,3,6.750000,2023-01-10T19:59:59.999000,10103
1,1010,35.570470,300,8,12.416667,10.25,8,12.416667,2023-01-10T19:59:59.999000,10109
2,1010,66.666667,300,3,9.000000,8.00,3,9.000000,2023-01-10T19:59:59.999000,101010
3,1015,41.666667,300,7,12.000000,11.50,7,12.000000,2023-01-10T19:59:59.999000,10152
4,1015,25.000000,300,6,8.000000,8.00,6,8.000000,2023-01-10T19:59:59.999000,10158
5,1020,14.792899,300,18,21.125000,22.25,18,21.125000,2023-01-10T19:54:59.999000,10207
6,1020,19.018405,300,11,13.583333,13.25,11,13.583333,2023-01-10T19:54:59.999000,10208
7,1395,10.489510,300,16,17.875000,17.25,16,17.875000,2023-01-10T19:54:59.999000,139529
8,1395,37.931034,300,9,14.500000,15.25,9,14.500000,2023-01-10T19:54:59.999000,139531
9,2870,21.495327,300,7,8.916667,7.75,7,8.916667,2023-01-10T19:54:59.999000,28702


In [29]:
from ksql_data_helpers import return_query_df

In [52]:
def return_det_counts_data(detector_id, time_start, time_end):
    '''
    detector_id: detector_id string (signalID + channel value)
    start_time: unix time in ms (INT)
    end_time: unix time in ms(INT)
    '''
    TABLE_NAME = 'COUNTS_RAW_STREAM_V2_AGG_5SEC' #Detector ID to counts  #THIS IS A TABLE
    ksql_string = f'SELECT * FROM {TABLE_NAME} where DETECTORID = ' +  "'" + detector_id + "'"   + f' AND WINDOWSTART>={time_start} AND WINDOWSTART < {time_end}'
    print(ksql_string)
    temp_df = return_query_df(ksql_string)
    
    temp_df.drop(columns = ['DETECTORID', 'WINDOWEND'], inplace = True)
    new_row = pd.Series(data={'WINDOWSTART':time_start,'COUNT':0}, name=0)
    temp_df = temp_df.append(new_row, ignore_index=False)
    new_row = pd.Series(data={'WINDOWSTART':time_end,'COUNT':0}, name=0)
    temp_df = temp_df.append(new_row, ignore_index=False)
    temp_df['timestamp']  = pd.to_datetime(temp_df['WINDOWSTART'], unit = 'ms').dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
    temp_df.set_index('timestamp', inplace = True)
    temp_df.drop(columns = ['WINDOWSTART'], inplace = True)
    temp_df.sort_index(inplace = True)

    final_df = temp_df.resample('5s').sum()
    
    
    return final_df


def return_det_counts_data_v2(detector_id, time_start, time_end):
    '''
    detector_id: detector_id string (signalID + channel value)
    start_time: unix time in ms (INT)
    end_time: unix time in ms(INT)
    '''
    TABLE_NAME = 'COUNTS_RAW_STREAM_V1' #Detector ID to counts  #THIS IS A TABLE
    ksql_string = f'SELECT * FROM {TABLE_NAME} where DETECTORID = ' +  "'" + detector_id + "'"   + f' AND TS>={time_start} AND TS < {time_end}'
    print(ksql_string)
    temp_df = return_query_df(ksql_string)
    temp_df.drop(columns = ['DETECTORID', 'SIGNALID'], inplace = True)
    temp_df['EVENTCODE'].replace([82], 1, inplace  = True)
    temp_df.rename(columns={'EVENTCODE': 'COUNT'},inplace  = True)
    
    
    new_row = pd.Series(data={'TS':time_start,'COUNT':0}, name=0)
    temp_df = temp_df.append(new_row, ignore_index=False)
    new_row = pd.Series(data={'TS':time_end,'COUNT':0}, name=0)
    temp_df = temp_df.append(new_row, ignore_index=False)
    temp_df['timestamp']  = pd.to_datetime(temp_df['TS'], unit = 'ms').dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
    temp_df.set_index('timestamp', inplace = True)
    temp_df.drop(columns = ['TS'], inplace = True)
    temp_df.sort_index(inplace = True)
    
    final_df = temp_df.resample('5s').sum()
    
    return final_df
    


In [53]:
detector_id = '10103'

time_start = 1673481600000
time_end = 1673483099999

In [56]:
temp_df = return_det_counts_data_v2(detector_id, time_start, time_end)

SELECT * FROM COUNTS_RAW_STREAM_V1 where DETECTORID = '10103' AND TS>=1673481600000 AND TS < 1673483099999


In [55]:
temp_df

,COUNT
timestamp,
2023-01-11 19:00:00-05:00,1
2023-01-11 19:00:05-05:00,0
2023-01-11 19:00:10-05:00,1
2023-01-11 19:00:15-05:00,0
2023-01-11 19:00:20-05:00,0
...,...
2023-01-11 19:24:35-05:00,0
2023-01-11 19:24:40-05:00,0
2023-01-11 19:24:45-05:00,0


In [45]:
temp_df.drop(columns = ['DETECTORID', 'SIGNALID'], inplace = True)
temp_df['EVENTCODE'].replace([82], 1, inplace  = True)
temp_df.rename(columns={'EVENTCODE': 'COUNT'},inplace  = True)

In [46]:
new_row = pd.Series(data={'TS':time_start,'COUNT':0}, name=0)
temp_df = temp_df.append(new_row, ignore_index=False)
new_row = pd.Series(data={'TS':time_end,'COUNT':0}, name=0)
temp_df = temp_df.append(new_row, ignore_index=False)
temp_df['timestamp']  = pd.to_datetime(temp_df['TS'], unit = 'ms').dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
temp_df.set_index('timestamp', inplace = True)
temp_df.drop(columns = ['TS'], inplace = True)
temp_df.sort_index(inplace = True)

In [49]:
final_df = temp_df.resample('5s').sum()

In [51]:
final_df['COUNT']

45

In [ ]:
temp_df.drop(columns = ['SIGNALID', 'DETECTORID'], inplace = True)

In [36]:
df

,SIGNALID,TS,EVENTCODE,DETECTORID
0,1010,1673481601000,82,10103
1,1010,1673481614000,82,10103
2,1010,1673481652000,82,10103
3,1010,1673481654000,82,10103
4,1010,1673481809000,82,10103
5,1010,1673481856000,82,10103
6,1010,1673481858000,82,10103
7,1010,1673481859000,82,10103
8,1010,1673481869000,82,10103
9,1010,1673481908000,82,10103


In [ ]:
 self.all_isc_store